In [6]:
import xgboost as xgb
from sklearn.datasets import load_svmlight_file

#  This script demonstrate how to do ranking with XGBRanker
x_train, y_train = load_svmlight_file("mq2008.train")
x_valid, y_valid = load_svmlight_file("mq2008.vali")
x_test, y_test = load_svmlight_file("mq2008.test")

display(x_train, y_train, x_valid, y_valid, x_test, y_test)

group_train = []
with open("mq2008.train.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_train.append(int(line.split("\n")[0]))

group_valid = []
with open("mq2008.vali.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_valid.append(int(line.split("\n")[0]))

params = {'objective': 'rank:ndcg', 'learning_rate': 0.1,
          'gamma': 1.0, 'min_child_weight': 0.1,
          'max_depth': 6, 'n_estimators': 4}

model = xgb.sklearn.XGBRanker(**params)
model.fit(x_train, y_train, group_train, verbose=True,
          eval_set=[(x_valid, y_valid)], eval_group=[group_valid])

pred = model.predict(x_test)

<9630x46 sparse matrix of type '<class 'numpy.float64'>'
	with 233775 stored elements in Compressed Sparse Row format>

array([0., 0., 0., ..., 0., 0., 0.])

<2707x46 sparse matrix of type '<class 'numpy.float64'>'
	with 64764 stored elements in Compressed Sparse Row format>

array([0., 0., 1., ..., 0., 0., 0.])

<2874x46 sparse matrix of type '<class 'numpy.float64'>'
	with 71241 stored elements in Compressed Sparse Row format>

array([0., 0., 0., ..., 1., 0., 0.])

[0]	eval_0-map:0.70380
[1]	eval_0-map:0.70857
[2]	eval_0-map:0.71028
[3]	eval_0-map:0.71069


In [20]:
import pandas as pd
import xgboost as xgb
from sklearn.datasets import load_svmlight_file

testdf = pd.read_pickle("../pickles/df_small_clean_test.pkl")
traindf = pd.read_pickle("../pickles/df_small_clean.pkl")
testdf = testdf.drop("PriceBand", axis=1)
traindf = traindf.drop("PriceBand", axis=1)

In [46]:
print(testdf.columns)
print(traindf.columns)

# split label and other variables
x_train, y_train = traindf.drop(columns=["position", "prop_id"]), traindf["position"]
x_test = testdf.drop(columns=["prop_id"])

params = {'objective': 'rank:ndcg', 'learning_rate': 0.1,
          'gamma': 1.0, 'min_child_weight': 0.1,
          'max_depth': 6, 'n_estimators': 4}

# TODO: ik snap niet precies wat de groups zijn in XGBRanker
query_lengths = []
[query_lengths.append(len(group)) for srchid, group in x_train.groupby('srch_id')]

model = xgb.sklearn.XGBRanker(**params)
model.fit(x_train, y_train, query_lengths, verbose=True)

pred = model.predict(x_train)

Index(['prop_id', 'srch_id', 'price_quality', 'competitor_bool',
       'availability_bools', 'visited_before', 'promotion_flag',
       'prop_brand_bool', 'random_bool', 'prop_location_score1',
       'prop_location_score2'],
      dtype='object')
Index(['prop_id', 'srch_id', 'position', 'price_quality', 'competitor_bool',
       'availability_bools', 'visited_before', 'click_bool', 'booking_bool',
       'promotion_flag', 'prop_brand_bool', 'random_bool',
       'prop_location_score1', 'prop_location_score2', 'importance'],
      dtype='object')


In [48]:
# prop_location_score2 en prop_brand_bool zijn belangrijke features
feature_importances = pd.DataFrame(x_train.columns, model.feature_importances_)

print(model.get_booster().get_score(importance_type='weight'))
feature_importances.head(14)

{'prop_location_score2': 49, 'price_quality': 56, 'srch_id': 50, 'random_bool': 7, 'prop_location_score1': 40, 'competitor_bool': 5, 'prop_brand_bool': 10, 'availability_bools': 7, 'promotion_flag': 9, 'click_bool': 1, 'visited_before': 1}


,0
0.084487,srch_id
0.052800,price_quality
0.015565,competitor_bool
0.033988,availability_bools
0.000091,visited_before
0.129069,click_bool
0.000000,booking_bool
0.070284,promotion_flag
0.212696,prop_brand_bool
0.014002,random_bool


In [43]:
x_train["predicted_rank"] = pred
x_train["position"] = y_train
predictions_sorted = []

for srchid, group in x_train.groupby('srch_id'):

    # TODO: predict position with randomforest
    predictions = [x for _,x in sorted(zip(group.predicted_rank, group.position))]
    predictions_sorted.append(predictions)


predictions_sorted = [item for sublist in predictions_sorted for item in sublist]
x_train["predicted_rank"] = predictions_sorted
x_train.to_pickle("../pickles/XGBoost_small_solution.pkl")